In [1]:
import pickle

# Open the pkl file
with open('regression_data.pkl', 'rb') as file:
    df_merged = pickle.load(file)

# Display the data
display(df_merged)


,week_day,day,bicycle_flux,cargo_bicycle_flux,moped_flux,scooter_flux,id,bicycle_min,bicycle_max,bicycle_diff,...,nearest_train_station_distance_driving,nearest_metro_station_name_driving,nearest_metro_station_distance_driving,nearest_train_station_name_transit,nearest_train_station_distance_transit,nearest_metro_station_name_transit,nearest_metro_station_distance_transit,population,highest_percentage_age,predicted_mean_disposable_household_income
Aetsveld/Oostelijke Vechtoever,sunday,0,0.0,0.0,0.0,0.0,Aetsveld/Oostelijke Vechtoever_0,0.0,0.0,0.0,...,11934,Gaasperplas,7997,Amsterdam Bijlmer ArenA Station,13299,Gaasperplas,8753.0,4402.0,27 - 65 years (%),39520.1
Amstel III/Bullewijk,sunday,0,0.0,0.0,36.0,0.0,Amstel III/Bullewijk_0,0.0,0.0,0.0,...,695,Station Holendrecht,941,Amsterdam Holendrecht Station,719,Station Holendrecht,560.0,3546.0,18 - 26 years (%),39520.1
Amsterdamse Poort e.o.,sunday,0,0.0,1.0,17.0,0.0,Amsterdamse Poort e.o._0,0.0,0.0,0.0,...,671,Station Bijlmer ArenA,671,Amsterdam Bijlmer ArenA Station,378,Station Bijlmer ArenA,378.0,8272.0,27 - 65 years (%),39520.1
Apollobuurt,sunday,0,7.0,0.0,46.0,0.0,Apollobuurt_0,2.0,5.0,3.0,...,1634,De Pijp,2145,Amsterdam Zuid Station,1902,Station Zuid,1253.0,8825.0,27 - 65 years (%),88559.5
Banne Buiksloot,sunday,0,0.0,0.0,0.0,0.0,Banne Buiksloot_0,0.0,0.0,0.0,...,10657,Noorderpark,1881,Amsterdam Centraal Station,4919,Noorderpark,1436.0,14806.0,27 - 65 years (%),42139.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westlandgracht,wednesday,31,5.0,0.0,1.0,0.0,Westlandgracht_31,14.0,19.0,5.0,...,1392,Henk Sneevlietweg,812,Amsterdam Lelylaan Station,1979,Henk Sneevlietweg,619.0,13194.0,27 - 65 years (%),46335.9
Willemspark,wednesday,31,0.0,0.0,0.0,0.0,Willemspark_31,0.0,0.0,0.0,...,2909,Heemstedestraat,2345,Amsterdam Lelylaan Station,2479,Amstelveenseweg,2084.0,5561.0,27 - 65 years (%),89532.3
Zeeburgereiland/Bovendiep,wednesday,31,0.0,0.0,0.0,0.0,Zeeburgereiland/Bovendiep_31,1.0,1.0,0.0,...,3977,Weesperplein,5785,Amsterdam Muiderpoort Station,2685,Wibautstraat,4889.0,5679.0,27 - 65 years (%),42204.7
Zuid Pijp,wednesday,31,0.0,0.0,1.0,0.0,Zuid Pijp_31,2.0,2.0,0.0,...,2404,De Pijp,0,Amsterdam Muiderpoort Station,2610,Wibautstraat,1275.0,7917.0,27 - 65 years (%),39976.9


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Assuming 'df' is your DataFrame
#df_merged = pd.get_dummies(df_merged, columns=['highest_percentage_age'], prefix='age_group')
# One-hot encode 'week_day'

# Feature Engineering
#df_merged['week_day'] = pd.Categorical(df_merged['week_day'])
#df_merged['day'] = pd.to_datetime(df_merged['day'])
#df_merged['day_of_week'] = df_merged['day'].dt.dayofweek
#df_merged['month'] = df_merged['day'].dt.month

df_merged['index_encoded'] = df_merged.index.astype(str)
df_merged = pd.get_dummies(df_merged, columns=['highest_percentage_age', 'week_day', 'nearest_train_station_name_walking', 'nearest_train_station_name_bicycling', 'nearest_train_station_name_driving', 'nearest_train_station_name_transit', 'nearest_metro_station_name_walking', 'nearest_metro_station_name_bicycling', 'nearest_metro_station_name_driving', 'nearest_metro_station_name_transit', 'index_encoded'], prefix=['age_group', 'day', 'walking', 'bicycling', 'driving', 'transit', 'walking', 'bicycling', 'driving', 'transit', 'index'])

#df_merged = pd.get_dummies(df_merged, columns=['index_encoded'], prefix='index')

# Create interaction terms
#df_merged['interaction_term_1'] = df_merged['bicycle_flux'] * df_merged['nearest_train_station_distance_driving']

# Additional Features (if available)
# Include relevant external data or weather-related features

# Selecting features and target variable
X = df_merged[['nearest_train_station_distance_walking', 'nearest_metro_station_distance_walking', 'nearest_train_station_distance_bicycling', 'nearest_metro_station_distance_bicycling', 'nearest_train_station_distance_driving', 'nearest_metro_station_distance_driving', 'nearest_train_station_distance_transit', 'nearest_metro_station_distance_transit', 
        'population',
        'predicted_mean_disposable_household_income', 
        'age_group_18 - 26 years (%)', 'age_group_27 - 65 years (%)', 'age_group_66 + years (%)',
        'day_monday', 'day_tuesday', 'day_wednesday', 'day_thursday', 'day_friday', 'day_saturday', 'day_sunday',
        'walking_Amsterdam Holendrecht Station', 'walking_Amsterdam Bijlmer ArenA Station', 'walking_Amsterdam Zuid Station', 'walking_Amsterdam Centraal Station', 'walking_Amsterdam Lelylaan Station', 'walking_Amsterdam Science Park Station', 'walking_Amsterdam RAI Station', 'walking_Amsterdam Muiderpoort Station', 'walking_Amsterdam Sloterdijk Station', 'walking_Amsterdam Duivendrecht Station',
        'bicycling_Amsterdam Holendrecht Station', 'bicycling_Amsterdam Bijlmer ArenA Station', 'bicycling_Amsterdam Zuid Station', 'bicycling_Amsterdam Centraal Station', 'bicycling_Amsterdam Lelylaan Station', 'bicycling_Amsterdam Science Park Station', 'bicycling_Amsterdam RAI Station', 'bicycling_Amsterdam Muiderpoort Station', 'bicycling_Amsterdam Sloterdijk Station', 'bicycling_Amsterdam Duivendrecht Station',
        'driving_Amsterdam Holendrecht Station', 'driving_Amsterdam Bijlmer ArenA Station', 'driving_Amsterdam Zuid Station', 'driving_Amsterdam Centraal Station', 'driving_Amsterdam Lelylaan Station', 'driving_Amsterdam Science Park Station', 'driving_Amsterdam RAI Station', 'driving_Amsterdam Muiderpoort Station', 'driving_Amsterdam Sloterdijk Station', 'driving_Amsterdam Duivendrecht Station',
        'transit_Amsterdam Holendrecht Station', 'transit_Amsterdam Bijlmer ArenA Station', 'transit_Amsterdam Zuid Station', 'transit_Amsterdam Centraal Station', 'transit_Amsterdam Lelylaan Station', 'transit_Amsterdam Science Park Station', 'transit_Amsterdam RAI Station', 'transit_Amsterdam Muiderpoort Station', 'transit_Amsterdam Sloterdijk Station', 'transit_Amsterdam Duivendrecht Station',
        'walking_Gaasperplas', 'walking_Station Holendrecht', 'walking_Station Bijlmer ArenA', 'walking_Station Zuid', 'walking_Noorderpark', 'walking_Postjesweg', 'walking_Station Diemen-Zuid', 'walking_Kraaiennest station', 'walking_Noord', 'walking_Station RAI', 'walking_Rokin', 'walking_Nieuwmarkt', 'walking_Jan van Galenstraat', 'walking_Station Lelylaan', 'walking_Burg. De Vlugtlaan', 'walking_Vijzelgracht', 'walking_Amstelstation', 'walking_Station Ganzenhoef', 'walking_Gein', 'walking_Waterlooplein', 'walking_Centraal Station', 'walking_Heemstedestraat', 'walking_Europaplein', 'walking_Amstelveenseweg', 'walking_Henk Sneevlietweg', 'walking_Station Sloterdijk', 'walking_Reigersbos', 'walking_Isolatorweg', 'walking_Venserpolder', 'walking_Wibautstraat', 'walking_De Pijp',
        'bicycling_Gaasperplas', 'bicycling_Station Holendrecht', 'bicycling_Station Bijlmer ArenA', 'bicycling_Station Zuid', 'bicycling_Noorderpark', 'bicycling_Postjesweg', 'bicycling_Kraaiennest station', 'bicycling_Noord', 'bicycling_Rokin', 'bicycling_Nieuwmarkt', 'bicycling_Jan van Galenstraat', 'bicycling_Station Lelylaan', 'bicycling_Burg. De Vlugtlaan', 'bicycling_Vijzelgracht', 'bicycling_Amstelstation', 'bicycling_Station Ganzenhoef', 'bicycling_Gein', 'bicycling_Waterlooplein', 'bicycling_Centraal Station', 'bicycling_Heemstedestraat', 'bicycling_Europaplein', 'bicycling_Amstelveenseweg', 'bicycling_Henk Sneevlietweg', 'bicycling_Station Sloterdijk', 'bicycling_Reigersbos', 'bicycling_Isolatorweg', 'bicycling_Venserpolder', 'bicycling_Wibautstraat', 'bicycling_De Pijp',
        'driving_Gaasperplas', 'driving_Station Holendrecht', 'driving_Station Bijlmer ArenA', 'driving_Noorderpark', 'driving_Postjesweg', 'driving_Kraaiennest station', 'driving_Noord', 'driving_Station RAI', 'driving_Nieuwmarkt', 'driving_Jan van Galenstraat', 'driving_Station Lelylaan', 'driving_Burg. De Vlugtlaan', 'driving_Vijzelgracht', 'driving_Amstelstation', 'driving_Station Ganzenhoef', 'driving_Gein', 'driving_Waterlooplein', 'driving_Centraal Station', 'driving_Heemstedestraat', 'driving_Europaplein', 'driving_Amstelveenseweg', 'driving_Henk Sneevlietweg', 'driving_Station Sloterdijk', 'driving_Reigersbos', 'driving_Isolatorweg', 'driving_Venserpolder', 'driving_Wibautstraat', 'driving_De Pijp',
        'transit_Gaasperplas', 'transit_Station Holendrecht', 'transit_Station Bijlmer ArenA', 'transit_Station Zuid', 'transit_Noorderpark', 'transit_Postjesweg', 'transit_Station Diemen-Zuid', 'transit_Kraaiennest station', 'transit_Noord', 'transit_Station RAI', 'transit_Rokin', 'transit_Nieuwmarkt', 'transit_Jan van Galenstraat', 'transit_Station Lelylaan', 'transit_Burg. De Vlugtlaan', 'transit_Vijzelgracht', 'transit_Amstelstation', 'transit_Station Ganzenhoef', 'transit_Waterlooplein', 'transit_Centraal Station', 'transit_Heemstedestraat', 'transit_Europaplein', 'transit_Amstelveenseweg', 'transit_Henk Sneevlietweg', 'transit_Station Sloterdijk', 'transit_Reigersbos', 'transit_Isolatorweg', 'transit_Wibautstraat', 'transit_De Pijp',
        'index_Aetsveld/Oostelijke Vechtoever', 'index_Amstel III/Bullewijk', 'index_Amsterdamse Poort e.o.', 'index_Apollobuurt', 'index_Banne Buiksloot', 'index_Bellamybuurt', 'index_Betondorp', 'index_Bijlmermuseum', 'index_Bloemendalerpolder', 'index_Buikslotermeer', 'index_Buitenveldert-Oost', 'index_Buitenveldert-West', 'index_Burgwallen-Nieuwe Zijde', 'index_Burgwallen-Oude Zijde', 'index_Centrale Markt', 'index_Chassébuurt', 'index_Coenhaven/Minervahaven', 'index_Da Costabuurt', 'index_Dapperbuurt', 'index_De Aker', 'index_De Kolenkit', 'index_De Punt', 'index_De Weteringschans', 'index_Driemond', 'index_Elzenhagen', 'index_Erasmuspark', 'index_Frankendael', 'index_Frederik Hendrikbuurt', 'index_Ganzenhoef e.o.', 'index_Geerdinkhof/Kantershof', 'index_Gein', 'index_Geuzenbuurt', 'index_Geuzenveld', 'index_Grachtengordel-West', 'index_Grachtengordel-Zuid', 'index_H-buurt', 'index_Haarlemmerbuurt', 'index_Havens-West', 'index_Helmersbuurt', 'index_Holendrecht', 'index_Hoofddorppleinbuurt', 'index_Hoofdweg e.o.', 'index_Houthavens', 'index_IJburg-Oost', 'index_IJburg-West', 'index_IJburg-Zuid', 'index_IJplein/Vogelbuurt', 'index_IJselbuurt', 'index_Indische Buurt-Oost', 'index_Indische Buurt-West', 'index_Jordaan', 'index_K-buurt', 'index_Kadoelen', 'index_Landlust', 'index_Lutkemeer/Ookmeer', 'index_Middenmeer', 'index_Museumkwartier', 'index_Nellestein', 'index_Nieuwe Pijp', 'index_Nieuwendammerdijk/Buiksloterdijk', 'index_Nieuwmarkt/Lastage', 'index_Noordelijke IJ-oevers-Oost', 'index_Noordelijke IJ-oevers-West', 'index_Omval/Overamstel', 'index_Oostelijk Havengebied', 'index_Oostelijke Eilanden/Kadijken', 'index_Oosterparkbuurt', 'index_Oostzanerwerf', 'index_Osdorp-Midden', 'index_Osdorp-Oost', 'index_Oude Pijp', 'index_Overtoomse Sluis', 'index_Overtoomse Veld', 'index_Prinses Irenebuurt e.o.', 'index_Reigersbos', 'index_Rijnbuurt', 'index_Scheldebuurt', 'index_Schinkelbuurt', 'index_Sloten/Nieuw-Sloten', 'index_Sloterdijk Nieuw-West', 'index_Sloterdijk-West', 'index_Slotermeer-Noordoost', 'index_Slotermeer-West', 'index_Slotermeer-Zuidoost', 'index_Slotervaart-Noord', 'index_Slotervaart-Zuid', 'index_Spaarndammerbuurt/Zeeheldenbuurt', 'index_Staatsliedenbuurt', 'index_Stadionbuurt', 'index_Transvaalbuurt', 'index_Tuindorp Buiksloot', 'index_Tuindorp Nieuwendam', 'index_Tuindorp Oostzaan', 'index_Van Galenbuurt', 'index_Van Lennepbuurt', 'index_Venserpolder', 'index_Volewijck', 'index_Vondelparkbuurt', 'index_Waterland', 'index_Waterlandpleinbuurt', 'index_Weesp Binnenstad/Zuid', 'index_Weesp-Noordwest', 'index_Weesperbuurt/Plantage', 'index_Weesperzijde', 'index_Westindische Buurt', 'index_Westlandgracht', 'index_Willemspark', 'index_Zeeburgereiland/Bovendiep', 'index_Zuid Pijp', 'index_Zuidas'
]]

y = df_merged[['bicycle_flux', 'cargo_bicycle_flux', 'moped_flux', 'scooter_flux']]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Selection and Hyperparameter Tuning (Random Forest)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

best_rf_model = grid_search.best_estimator_

# Model Evaluation
y_pred = best_rf_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature Importance Analysis
feature_importance = best_rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print('Feature Importance:')
print(feature_importance_df)

feature_importance_df.to_csv('feature_importance_df_flux.csv', index=False)


Mean Squared Error: 31.904389684966585
R-squared: 0.8345570516923142
Feature Importance:
                                        Feature  Importance
4        nearest_train_station_distance_driving    0.183303
9    predicted_mean_disposable_household_income    0.133073
2      nearest_train_station_distance_bicycling    0.100885
8                                    population    0.081368
0        nearest_train_station_distance_walking    0.076135
..                                          ...         ...
228                               index_K-buurt    0.000000
200                              index_Driemond    0.000000
205                       index_Ganzenhoef e.o.    0.000000
234                            index_Nellestein    0.000000
143                          driving_Reigersbos    0.000000

[287 rows x 2 columns]


In [3]:
# Selecting features and target variable
X = df_merged[['nearest_train_station_distance_walking', 'nearest_metro_station_distance_walking', 'nearest_train_station_distance_bicycling', 'nearest_metro_station_distance_bicycling', 'nearest_train_station_distance_driving', 'nearest_metro_station_distance_driving', 'nearest_train_station_distance_transit', 'nearest_metro_station_distance_transit', 
        'population',
        'predicted_mean_disposable_household_income', 
        'age_group_18 - 26 years (%)', 'age_group_27 - 65 years (%)', 'age_group_66 + years (%)',
        'day_monday', 'day_tuesday', 'day_wednesday', 'day_thursday', 'day_friday', 'day_saturday', 'day_sunday',
        'walking_Amsterdam Holendrecht Station', 'walking_Amsterdam Bijlmer ArenA Station', 'walking_Amsterdam Zuid Station', 'walking_Amsterdam Centraal Station', 'walking_Amsterdam Lelylaan Station', 'walking_Amsterdam Science Park Station', 'walking_Amsterdam RAI Station', 'walking_Amsterdam Muiderpoort Station', 'walking_Amsterdam Sloterdijk Station', 'walking_Amsterdam Duivendrecht Station',
        'bicycling_Amsterdam Holendrecht Station', 'bicycling_Amsterdam Bijlmer ArenA Station', 'bicycling_Amsterdam Zuid Station', 'bicycling_Amsterdam Centraal Station', 'bicycling_Amsterdam Lelylaan Station', 'bicycling_Amsterdam Science Park Station', 'bicycling_Amsterdam RAI Station', 'bicycling_Amsterdam Muiderpoort Station', 'bicycling_Amsterdam Sloterdijk Station', 'bicycling_Amsterdam Duivendrecht Station',
        'driving_Amsterdam Holendrecht Station', 'driving_Amsterdam Bijlmer ArenA Station', 'driving_Amsterdam Zuid Station', 'driving_Amsterdam Centraal Station', 'driving_Amsterdam Lelylaan Station', 'driving_Amsterdam Science Park Station', 'driving_Amsterdam RAI Station', 'driving_Amsterdam Muiderpoort Station', 'driving_Amsterdam Sloterdijk Station', 'driving_Amsterdam Duivendrecht Station',
        'transit_Amsterdam Holendrecht Station', 'transit_Amsterdam Bijlmer ArenA Station', 'transit_Amsterdam Zuid Station', 'transit_Amsterdam Centraal Station', 'transit_Amsterdam Lelylaan Station', 'transit_Amsterdam Science Park Station', 'transit_Amsterdam RAI Station', 'transit_Amsterdam Muiderpoort Station', 'transit_Amsterdam Sloterdijk Station', 'transit_Amsterdam Duivendrecht Station',
        'walking_Gaasperplas', 'walking_Station Holendrecht', 'walking_Station Bijlmer ArenA', 'walking_Station Zuid', 'walking_Noorderpark', 'walking_Postjesweg', 'walking_Station Diemen-Zuid', 'walking_Kraaiennest station', 'walking_Noord', 'walking_Station RAI', 'walking_Rokin', 'walking_Nieuwmarkt', 'walking_Jan van Galenstraat', 'walking_Station Lelylaan', 'walking_Burg. De Vlugtlaan', 'walking_Vijzelgracht', 'walking_Amstelstation', 'walking_Station Ganzenhoef', 'walking_Gein', 'walking_Waterlooplein', 'walking_Centraal Station', 'walking_Heemstedestraat', 'walking_Europaplein', 'walking_Amstelveenseweg', 'walking_Henk Sneevlietweg', 'walking_Station Sloterdijk', 'walking_Reigersbos', 'walking_Isolatorweg', 'walking_Venserpolder', 'walking_Wibautstraat', 'walking_De Pijp',
        'bicycling_Gaasperplas', 'bicycling_Station Holendrecht', 'bicycling_Station Bijlmer ArenA', 'bicycling_Station Zuid', 'bicycling_Noorderpark', 'bicycling_Postjesweg', 'bicycling_Kraaiennest station', 'bicycling_Noord', 'bicycling_Rokin', 'bicycling_Nieuwmarkt', 'bicycling_Jan van Galenstraat', 'bicycling_Station Lelylaan', 'bicycling_Burg. De Vlugtlaan', 'bicycling_Vijzelgracht', 'bicycling_Amstelstation', 'bicycling_Station Ganzenhoef', 'bicycling_Gein', 'bicycling_Waterlooplein', 'bicycling_Centraal Station', 'bicycling_Heemstedestraat', 'bicycling_Europaplein', 'bicycling_Amstelveenseweg', 'bicycling_Henk Sneevlietweg', 'bicycling_Station Sloterdijk', 'bicycling_Reigersbos', 'bicycling_Isolatorweg', 'bicycling_Venserpolder', 'bicycling_Wibautstraat', 'bicycling_De Pijp',
        'driving_Gaasperplas', 'driving_Station Holendrecht', 'driving_Station Bijlmer ArenA', 'driving_Noorderpark', 'driving_Postjesweg', 'driving_Kraaiennest station', 'driving_Noord', 'driving_Station RAI', 'driving_Nieuwmarkt', 'driving_Jan van Galenstraat', 'driving_Station Lelylaan', 'driving_Burg. De Vlugtlaan', 'driving_Vijzelgracht', 'driving_Amstelstation', 'driving_Station Ganzenhoef', 'driving_Gein', 'driving_Waterlooplein', 'driving_Centraal Station', 'driving_Heemstedestraat', 'driving_Europaplein', 'driving_Amstelveenseweg', 'driving_Henk Sneevlietweg', 'driving_Station Sloterdijk', 'driving_Reigersbos', 'driving_Isolatorweg', 'driving_Venserpolder', 'driving_Wibautstraat', 'driving_De Pijp',
        'transit_Gaasperplas', 'transit_Station Holendrecht', 'transit_Station Bijlmer ArenA', 'transit_Station Zuid', 'transit_Noorderpark', 'transit_Postjesweg', 'transit_Station Diemen-Zuid', 'transit_Kraaiennest station', 'transit_Noord', 'transit_Station RAI', 'transit_Rokin', 'transit_Nieuwmarkt', 'transit_Jan van Galenstraat', 'transit_Station Lelylaan', 'transit_Burg. De Vlugtlaan', 'transit_Vijzelgracht', 'transit_Amstelstation', 'transit_Station Ganzenhoef', 'transit_Waterlooplein', 'transit_Centraal Station', 'transit_Heemstedestraat', 'transit_Europaplein', 'transit_Amstelveenseweg', 'transit_Henk Sneevlietweg', 'transit_Station Sloterdijk', 'transit_Reigersbos', 'transit_Isolatorweg', 'transit_Wibautstraat', 'transit_De Pijp',
        'index_Aetsveld/Oostelijke Vechtoever', 'index_Amstel III/Bullewijk', 'index_Amsterdamse Poort e.o.', 'index_Apollobuurt', 'index_Banne Buiksloot', 'index_Bellamybuurt', 'index_Betondorp', 'index_Bijlmermuseum', 'index_Bloemendalerpolder', 'index_Buikslotermeer', 'index_Buitenveldert-Oost', 'index_Buitenveldert-West', 'index_Burgwallen-Nieuwe Zijde', 'index_Burgwallen-Oude Zijde', 'index_Centrale Markt', 'index_Chassébuurt', 'index_Coenhaven/Minervahaven', 'index_Da Costabuurt', 'index_Dapperbuurt', 'index_De Aker', 'index_De Kolenkit', 'index_De Punt', 'index_De Weteringschans', 'index_Driemond', 'index_Elzenhagen', 'index_Erasmuspark', 'index_Frankendael', 'index_Frederik Hendrikbuurt', 'index_Ganzenhoef e.o.', 'index_Geerdinkhof/Kantershof', 'index_Gein', 'index_Geuzenbuurt', 'index_Geuzenveld', 'index_Grachtengordel-West', 'index_Grachtengordel-Zuid', 'index_H-buurt', 'index_Haarlemmerbuurt', 'index_Havens-West', 'index_Helmersbuurt', 'index_Holendrecht', 'index_Hoofddorppleinbuurt', 'index_Hoofdweg e.o.', 'index_Houthavens', 'index_IJburg-Oost', 'index_IJburg-West', 'index_IJburg-Zuid', 'index_IJplein/Vogelbuurt', 'index_IJselbuurt', 'index_Indische Buurt-Oost', 'index_Indische Buurt-West', 'index_Jordaan', 'index_K-buurt', 'index_Kadoelen', 'index_Landlust', 'index_Lutkemeer/Ookmeer', 'index_Middenmeer', 'index_Museumkwartier', 'index_Nellestein', 'index_Nieuwe Pijp', 'index_Nieuwendammerdijk/Buiksloterdijk', 'index_Nieuwmarkt/Lastage', 'index_Noordelijke IJ-oevers-Oost', 'index_Noordelijke IJ-oevers-West', 'index_Omval/Overamstel', 'index_Oostelijk Havengebied', 'index_Oostelijke Eilanden/Kadijken', 'index_Oosterparkbuurt', 'index_Oostzanerwerf', 'index_Osdorp-Midden', 'index_Osdorp-Oost', 'index_Oude Pijp', 'index_Overtoomse Sluis', 'index_Overtoomse Veld', 'index_Prinses Irenebuurt e.o.', 'index_Reigersbos', 'index_Rijnbuurt', 'index_Scheldebuurt', 'index_Schinkelbuurt', 'index_Sloten/Nieuw-Sloten', 'index_Sloterdijk Nieuw-West', 'index_Sloterdijk-West', 'index_Slotermeer-Noordoost', 'index_Slotermeer-West', 'index_Slotermeer-Zuidoost', 'index_Slotervaart-Noord', 'index_Slotervaart-Zuid', 'index_Spaarndammerbuurt/Zeeheldenbuurt', 'index_Staatsliedenbuurt', 'index_Stadionbuurt', 'index_Transvaalbuurt', 'index_Tuindorp Buiksloot', 'index_Tuindorp Nieuwendam', 'index_Tuindorp Oostzaan', 'index_Van Galenbuurt', 'index_Van Lennepbuurt', 'index_Venserpolder', 'index_Volewijck', 'index_Vondelparkbuurt', 'index_Waterland', 'index_Waterlandpleinbuurt', 'index_Weesp Binnenstad/Zuid', 'index_Weesp-Noordwest', 'index_Weesperbuurt/Plantage', 'index_Weesperzijde', 'index_Westindische Buurt', 'index_Westlandgracht', 'index_Willemspark', 'index_Zeeburgereiland/Bovendiep', 'index_Zuid Pijp', 'index_Zuidas'
]]

y = df_merged[['bicycle_min', 'bicycle_max', 'bicycle_diff', 'cargo_bicycle_min', 'cargo_bicycle_max', 'cargo_bicycle_diff', 'moped_min', 'moped_max', 'moped_diff', 'scooter_min', 'scooter_max', 'scooter_diff']]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Selection and Hyperparameter Tuning (Random Forest)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

best_rf_model = grid_search.best_estimator_

# Model Evaluation
y_pred = best_rf_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature Importance Analysis
feature_importance = best_rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print('Feature Importance:')
print(feature_importance_df)

feature_importance_df.to_csv('feature_importance_df_min_max_diff.csv', index=False)

Mean Squared Error: 6.471245354481199
R-squared: 0.8436168775955813
Feature Importance:
                                        Feature  Importance
8                                    population    0.141205
4        nearest_train_station_distance_driving    0.106593
189               index_Burgwallen-Nieuwe Zijde    0.090477
9    predicted_mean_disposable_household_income    0.076786
6        nearest_train_station_distance_transit    0.063554
..                                          ...         ...
60                          walking_Gaasperplas    0.000000
106                bicycling_Station Ganzenhoef    0.000000
234                            index_Nellestein    0.000000
97                bicycling_Kraaiennest station    0.000000
143                          driving_Reigersbos    0.000000

[287 rows x 2 columns]


In [4]:
# Selecting features and target variable
X = df_merged[['nearest_train_station_distance_walking', 'nearest_metro_station_distance_walking', 'nearest_train_station_distance_bicycling', 'nearest_metro_station_distance_bicycling', 'nearest_train_station_distance_driving', 'nearest_metro_station_distance_driving', 'nearest_train_station_distance_transit', 'nearest_metro_station_distance_transit', 
        'population',
        'predicted_mean_disposable_household_income', 
        'age_group_18 - 26 years (%)', 'age_group_27 - 65 years (%)', 'age_group_66 + years (%)',
        'day_monday', 'day_tuesday', 'day_wednesday', 'day_thursday', 'day_friday', 'day_saturday', 'day_sunday',
        'walking_Amsterdam Holendrecht Station', 'walking_Amsterdam Bijlmer ArenA Station', 'walking_Amsterdam Zuid Station', 'walking_Amsterdam Centraal Station', 'walking_Amsterdam Lelylaan Station', 'walking_Amsterdam Science Park Station', 'walking_Amsterdam RAI Station', 'walking_Amsterdam Muiderpoort Station', 'walking_Amsterdam Sloterdijk Station', 'walking_Amsterdam Duivendrecht Station',
        'bicycling_Amsterdam Holendrecht Station', 'bicycling_Amsterdam Bijlmer ArenA Station', 'bicycling_Amsterdam Zuid Station', 'bicycling_Amsterdam Centraal Station', 'bicycling_Amsterdam Lelylaan Station', 'bicycling_Amsterdam Science Park Station', 'bicycling_Amsterdam RAI Station', 'bicycling_Amsterdam Muiderpoort Station', 'bicycling_Amsterdam Sloterdijk Station', 'bicycling_Amsterdam Duivendrecht Station',
        'driving_Amsterdam Holendrecht Station', 'driving_Amsterdam Bijlmer ArenA Station', 'driving_Amsterdam Zuid Station', 'driving_Amsterdam Centraal Station', 'driving_Amsterdam Lelylaan Station', 'driving_Amsterdam Science Park Station', 'driving_Amsterdam RAI Station', 'driving_Amsterdam Muiderpoort Station', 'driving_Amsterdam Sloterdijk Station', 'driving_Amsterdam Duivendrecht Station',
        'transit_Amsterdam Holendrecht Station', 'transit_Amsterdam Bijlmer ArenA Station', 'transit_Amsterdam Zuid Station', 'transit_Amsterdam Centraal Station', 'transit_Amsterdam Lelylaan Station', 'transit_Amsterdam Science Park Station', 'transit_Amsterdam RAI Station', 'transit_Amsterdam Muiderpoort Station', 'transit_Amsterdam Sloterdijk Station', 'transit_Amsterdam Duivendrecht Station',
        'walking_Gaasperplas', 'walking_Station Holendrecht', 'walking_Station Bijlmer ArenA', 'walking_Station Zuid', 'walking_Noorderpark', 'walking_Postjesweg', 'walking_Station Diemen-Zuid', 'walking_Kraaiennest station', 'walking_Noord', 'walking_Station RAI', 'walking_Rokin', 'walking_Nieuwmarkt', 'walking_Jan van Galenstraat', 'walking_Station Lelylaan', 'walking_Burg. De Vlugtlaan', 'walking_Vijzelgracht', 'walking_Amstelstation', 'walking_Station Ganzenhoef', 'walking_Gein', 'walking_Waterlooplein', 'walking_Centraal Station', 'walking_Heemstedestraat', 'walking_Europaplein', 'walking_Amstelveenseweg', 'walking_Henk Sneevlietweg', 'walking_Station Sloterdijk', 'walking_Reigersbos', 'walking_Isolatorweg', 'walking_Venserpolder', 'walking_Wibautstraat', 'walking_De Pijp',
        'bicycling_Gaasperplas', 'bicycling_Station Holendrecht', 'bicycling_Station Bijlmer ArenA', 'bicycling_Station Zuid', 'bicycling_Noorderpark', 'bicycling_Postjesweg', 'bicycling_Kraaiennest station', 'bicycling_Noord', 'bicycling_Rokin', 'bicycling_Nieuwmarkt', 'bicycling_Jan van Galenstraat', 'bicycling_Station Lelylaan', 'bicycling_Burg. De Vlugtlaan', 'bicycling_Vijzelgracht', 'bicycling_Amstelstation', 'bicycling_Station Ganzenhoef', 'bicycling_Gein', 'bicycling_Waterlooplein', 'bicycling_Centraal Station', 'bicycling_Heemstedestraat', 'bicycling_Europaplein', 'bicycling_Amstelveenseweg', 'bicycling_Henk Sneevlietweg', 'bicycling_Station Sloterdijk', 'bicycling_Reigersbos', 'bicycling_Isolatorweg', 'bicycling_Venserpolder', 'bicycling_Wibautstraat', 'bicycling_De Pijp',
        'driving_Gaasperplas', 'driving_Station Holendrecht', 'driving_Station Bijlmer ArenA', 'driving_Noorderpark', 'driving_Postjesweg', 'driving_Kraaiennest station', 'driving_Noord', 'driving_Station RAI', 'driving_Nieuwmarkt', 'driving_Jan van Galenstraat', 'driving_Station Lelylaan', 'driving_Burg. De Vlugtlaan', 'driving_Vijzelgracht', 'driving_Amstelstation', 'driving_Station Ganzenhoef', 'driving_Gein', 'driving_Waterlooplein', 'driving_Centraal Station', 'driving_Heemstedestraat', 'driving_Europaplein', 'driving_Amstelveenseweg', 'driving_Henk Sneevlietweg', 'driving_Station Sloterdijk', 'driving_Reigersbos', 'driving_Isolatorweg', 'driving_Venserpolder', 'driving_Wibautstraat', 'driving_De Pijp',
        'transit_Gaasperplas', 'transit_Station Holendrecht', 'transit_Station Bijlmer ArenA', 'transit_Station Zuid', 'transit_Noorderpark', 'transit_Postjesweg', 'transit_Station Diemen-Zuid', 'transit_Kraaiennest station', 'transit_Noord', 'transit_Station RAI', 'transit_Rokin', 'transit_Nieuwmarkt', 'transit_Jan van Galenstraat', 'transit_Station Lelylaan', 'transit_Burg. De Vlugtlaan', 'transit_Vijzelgracht', 'transit_Amstelstation', 'transit_Station Ganzenhoef', 'transit_Waterlooplein', 'transit_Centraal Station', 'transit_Heemstedestraat', 'transit_Europaplein', 'transit_Amstelveenseweg', 'transit_Henk Sneevlietweg', 'transit_Station Sloterdijk', 'transit_Reigersbos', 'transit_Isolatorweg', 'transit_Wibautstraat', 'transit_De Pijp',
        'index_Aetsveld/Oostelijke Vechtoever', 'index_Amstel III/Bullewijk', 'index_Amsterdamse Poort e.o.', 'index_Apollobuurt', 'index_Banne Buiksloot', 'index_Bellamybuurt', 'index_Betondorp', 'index_Bijlmermuseum', 'index_Bloemendalerpolder', 'index_Buikslotermeer', 'index_Buitenveldert-Oost', 'index_Buitenveldert-West', 'index_Burgwallen-Nieuwe Zijde', 'index_Burgwallen-Oude Zijde', 'index_Centrale Markt', 'index_Chassébuurt', 'index_Coenhaven/Minervahaven', 'index_Da Costabuurt', 'index_Dapperbuurt', 'index_De Aker', 'index_De Kolenkit', 'index_De Punt', 'index_De Weteringschans', 'index_Driemond', 'index_Elzenhagen', 'index_Erasmuspark', 'index_Frankendael', 'index_Frederik Hendrikbuurt', 'index_Ganzenhoef e.o.', 'index_Geerdinkhof/Kantershof', 'index_Gein', 'index_Geuzenbuurt', 'index_Geuzenveld', 'index_Grachtengordel-West', 'index_Grachtengordel-Zuid', 'index_H-buurt', 'index_Haarlemmerbuurt', 'index_Havens-West', 'index_Helmersbuurt', 'index_Holendrecht', 'index_Hoofddorppleinbuurt', 'index_Hoofdweg e.o.', 'index_Houthavens', 'index_IJburg-Oost', 'index_IJburg-West', 'index_IJburg-Zuid', 'index_IJplein/Vogelbuurt', 'index_IJselbuurt', 'index_Indische Buurt-Oost', 'index_Indische Buurt-West', 'index_Jordaan', 'index_K-buurt', 'index_Kadoelen', 'index_Landlust', 'index_Lutkemeer/Ookmeer', 'index_Middenmeer', 'index_Museumkwartier', 'index_Nellestein', 'index_Nieuwe Pijp', 'index_Nieuwendammerdijk/Buiksloterdijk', 'index_Nieuwmarkt/Lastage', 'index_Noordelijke IJ-oevers-Oost', 'index_Noordelijke IJ-oevers-West', 'index_Omval/Overamstel', 'index_Oostelijk Havengebied', 'index_Oostelijke Eilanden/Kadijken', 'index_Oosterparkbuurt', 'index_Oostzanerwerf', 'index_Osdorp-Midden', 'index_Osdorp-Oost', 'index_Oude Pijp', 'index_Overtoomse Sluis', 'index_Overtoomse Veld', 'index_Prinses Irenebuurt e.o.', 'index_Reigersbos', 'index_Rijnbuurt', 'index_Scheldebuurt', 'index_Schinkelbuurt', 'index_Sloten/Nieuw-Sloten', 'index_Sloterdijk Nieuw-West', 'index_Sloterdijk-West', 'index_Slotermeer-Noordoost', 'index_Slotermeer-West', 'index_Slotermeer-Zuidoost', 'index_Slotervaart-Noord', 'index_Slotervaart-Zuid', 'index_Spaarndammerbuurt/Zeeheldenbuurt', 'index_Staatsliedenbuurt', 'index_Stadionbuurt', 'index_Transvaalbuurt', 'index_Tuindorp Buiksloot', 'index_Tuindorp Nieuwendam', 'index_Tuindorp Oostzaan', 'index_Van Galenbuurt', 'index_Van Lennepbuurt', 'index_Venserpolder', 'index_Volewijck', 'index_Vondelparkbuurt', 'index_Waterland', 'index_Waterlandpleinbuurt', 'index_Weesp Binnenstad/Zuid', 'index_Weesp-Noordwest', 'index_Weesperbuurt/Plantage', 'index_Weesperzijde', 'index_Westindische Buurt', 'index_Westlandgracht', 'index_Willemspark', 'index_Zeeburgereiland/Bovendiep', 'index_Zuid Pijp', 'index_Zuidas'
]]

y = df_merged[['bicycle_avg', 'cargo_bicycle_avg', 'moped_avg', 'scooter_avg']]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Selection and Hyperparameter Tuning (Random Forest)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

best_rf_model = grid_search.best_estimator_

# Model Evaluation
y_pred = best_rf_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Feature Importance Analysis
feature_importance = best_rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print('Feature Importance:')
print(feature_importance_df)

feature_importance_df.to_csv('feature_importance_avg_parking.csv', index=False)

Mean Squared Error: 4.394957541600121
R-squared: 0.9077995452670594
Feature Importance:
                                        Feature  Importance
8                                    population    0.215014
189               index_Burgwallen-Nieuwe Zijde    0.112574
6        nearest_train_station_distance_transit    0.077893
9    predicted_mean_disposable_household_income    0.076093
286                                index_Zuidas    0.057481
..                                          ...         ...
278                       index_Weesp-Noordwest    0.000000
200                              index_Driemond    0.000000
205                       index_Ganzenhoef e.o.    0.000000
228                               index_K-buurt    0.000000
148                         transit_Gaasperplas    0.000000

[287 rows x 2 columns]
